In [1]:
import pandas as pd
import numpy as np

from sklearn.covariance import EllipticEnvelope
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import plotly.express as px
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
train = pd.read_csv("data/train_data.csv")
test = pd.read_csv("data/test_data.csv")
train.head()

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe,type
0,1.59,41.00,0.7,20.53,1680.0,58.67,2.93,0
1,2.97,59.28,0.7,38.40,3142.0,74.91,3.75,0
2,1.91,45.29,0.7,24.73,2023.0,62.48,3.12,0
3,2.37,51.33,0.7,30.63,2506.0,67.84,3.39,0
4,1.90,45.21,0.7,24.65,2017.0,62.41,3.12,0


In [9]:
def get_pred_label(model_pred):
    # (1:정상, -1:불량) 이므로 (0:정상, 1:불량)로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

Model define & fit

In [7]:
train_x = train.copy()
test_x = test.copy()

In [3]:
val_contamination = 0.1 #휴리스틱하게 결정

In [6]:
model = EllipticEnvelope(support_fraction = 0.994, contamination = val_contamination, random_state = 42)
model.fit(train_x)

EllipticEnvelope(random_state=42, support_fraction=0.994)

In [10]:
test_pred = model.predict(test_x)
test_pred = get_pred_label(test_pred)
test['label'] = test_pred

In [11]:
test.head()

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe,type,label
0,2.51,53.28,0.7,32.54,2662.0,69.58,3.48,0,0
1,2.66,55.24,0.7,34.45,2819.0,71.32,3.57,0,0
2,1.72,42.74,0.7,22.23,1819.0,60.21,3.01,0,0
3,2.20,49.15,0.7,28.50,2332.0,65.91,3.30,0,0
4,2.06,47.28,0.7,26.67,2182.0,64.24,3.21,0,0


In [12]:
test['label'].value_counts()

0    6360
1    1029
Name: label, dtype: int64

In [13]:
answer = pd.read_csv("data/answer_sample.csv")
answer['label'] = test['label']

In [14]:
answer.to_csv("EllipticEnvelope.csv", index = False)